# Universidad de Buenos Aires

## Aprendizaje Profundo - TP3
## Cohorte 22 - 5to bimestre 2025

### Profesor: Esp. Ing. Gerardo Vilcamiza
### Alumno: Osvaldo Daniel Muñoz - SIU a2222

> **Formato de entrega:** un único notebook de Google Colab. Renombrar así: `MUNOZ-OSVALDO-DL-TP3-Co22.ipynb`.
> **Compartir con:** `gvilcamiza.ext@fi.uba.ar` con permisos de **comentador**.

Este notebook se puede ejecutar indistintamente en VS Code y en Colab.
Incluye:
- Las secciones y celdas ya organizadas por consigna.
- Cargar el dataset de imágenes en el entorno de Colab antes de ejecutar: (https://drive.google.com/file/d/1aPHE00zkDhEV1waJKhaOJMdN6-lUc0iT/view?usp=sharing)

### Objetivo general:

El objetivo de este trabajo es construir una red neuronal convolucional (CNN) utilizando Pytorch, capaz de clasificar emociones humanas a partir de imágenes faciales. El clasificador deberá identificar una de las 7 emociones básicas: alegría, tristeza, enojo, miedo, sorpresa, disgusto y seriedad.

### Punto 1. Preprocesamiento de Datos (2 puntos)
Antes de entrenar el modelo, se debe analizar qué tipo de preprocesamiento se debe aplicar a las imágenes. Para esto, se puede considerar uno o más aspectos como:

- Tamaño
- Relación de aspecto
- Color o escala de grises
- Cambio de dimensionalidad
- Normalización
- Balanceo de datos
- Data augmentation

Ser criteriosos y elegir solo las técnicas que consideren pertinentes para este caso de uso en específico.

Recomendación: usar torchvision.transforms para facilitar el preprocesamiento.

### Planteamiento inicial:

Para este trabajo analizaremos las características del dataset provisto, compuesto por imágenes de 100×100 píxeles, profundidad de color de 24 bits y distribución homogénea en RGB. Dado que el objetivo final incluye evaluar el modelo con imágenes externas (descargadas o tomadas de otras fuentes), se decide mantener la representación en RGB, evitando conversiones a escala de grises evitando eliminar información relevante sobre sombras y matices presentes en las imágenes reales.

Las imágenes fueron redimensionadas a un tamaño fijo de 100×100 píxeles, respetando la dimensionalidad original del dataset para evitar distorsiones y para mantener consistencia entre el conjunto de entrenamiento, validación y las imágenes externas utilizadas en pruebas adicionales.

Posteriormente aplicaremos normalización por canal, utilizando los valores estándar de ImageNet (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), lo cual facilita la estabilidad numérica durante el entrenamiento.

In [ ]:
# SETUP / IMPORTS / CONFIGURACIONES GENERALES

import os
# PyTorch
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# estilos para los plots
sns.set_theme(style="whitegrid", context="notebook")

# Reproducibilidad
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Mayor estabilidad a costa de velocidad en GPU
DETERMINISTIC = True

if DETERMINISTIC:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Device (CPU/GPU) según disponibilidad
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando device: {device}")

In [ ]:
# TRANSFORMACIONES

# Para entrenamiento
train_transforms = transforms.Compose([
    transforms.Resize((100, 100)),  # mantiene formato original del dataset
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # valores estándar RGB
        std=[0.229, 0.224, 0.225]
    )
])

# Para validación / test
val_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ===========================
# DATASETS
# ===========================

train_dir = "/home/ossiemunoz/projects/DL_TP3/dataset_emociones/train"
val_dir   = "/home/ossiemunoz/projects/DL_TP3/dataset_emociones/validation"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir,   transform=val_transforms)

# ===========================
# DATALOADERS
# ===========================

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False)

# Clases para referencia
class_names = train_dataset.classes
print("Clases:", class_names)

### Conclusiones preliminares del preprocesamiento:

Para mejorar la capacidad de generalización del modelo, incorporamos data augmentation controlado, limitado a transformaciones que no alteran la expresión facial:
- Rotaciones leves (≤10°), 
- RandomHorizontalFlip y 
- Variaciones moderadas de brillo/contraste. 

Evitamos transformaciones agresivas que podrían modificar la percepción de la emoción (rotación vertical, deformaciones geométricas grandes o cambios extremos de color).

Finalmente, se inspeccionó la distribución por clase para evaluar posibles desbalances. En caso de ser necesario, se prevé el uso de class weights o un WeightedRandomSampler para compensar diferencias significativas entre categorías.

Con este pipeline de preprocesamiento se garantiza que el modelo reciba entradas estandarizadas, robustas ante variaciones naturales del dominio y compatibles con imágenes externas que se utilizarán para validar su desempeño final.

In [2]:
# PATHS universales

# Detectamos si estamos en Google Colab
def is_colab():
    try:
        import google.colab  
        return True
    except ImportError:
        return False

if is_colab():
    from google.colab import drive  
    drive.mount('/content/drive')

    DATA_DIR = "/content/drive/MyDrive/DL_TP2"
    ENV = "Colab"
else:
    # Ruta local en el entorno VSC/WSL
    DATA_DIR = "/home/ossiemunoz/projects/DL_TP2"
    ENV = "Local (VSC/WSL)"

print(f"Entorno detectado: {ENV}")
print(f"DATA_DIR = {DATA_DIR}")

# Punto A - Análisis exploratorio de los datasets
- Realizar un EDA apoyado en gráficas adecuadas y coherentes para el caso de estudio.
- Analizar detalladamente los valores únicos de cada variable categórica e identificar su nivel de cardinalidad.
- Justificar de manera detallada el tipo de transformación que se le asignará a cada variable, en especial a las categóricas. **Dependiendo de su cardinalidad, su contexto y/o lógica interna de orden**, podrán transformarse mediante label/ordinal encoding, one-hot encoding o mediante una capa de embeddings dentro del modelo.
- No es necesario aplicar la misma transformación para todas las variables categóricas. El dataset puede (y debe) incluir diferentes tipos de transformaciones según las características de cada variable.
- Redactar explícitamente la decisión final adoptada para cada variable y su justificación correspondiente.

In [3]:
# A) Análisis exploratotio del dataset: Cargar datasets de entrenamiento y validación según paths universales

train_path = os.path.join(DATA_DIR, "adult_train.csv")
val_path   = os.path.join(DATA_DIR, "adult_val.csv")

df_train = pd.read_csv(train_path)
df_val   = pd.read_csv(val_path)

print(df_train.shape, df_val.shape)
df_train.info()
df_train.head(10)

In [4]:
# Limpieza básica

# 1) Normalizamos target a binario en train y val
df_train["income"] = df_train["income"].map({"<=50K": 0, ">50K": 1})
df_val["income"]   = df_val["income"].map({"<=50K": 0, ">50K": 1})

# 2) Reemplazar '?' por 'Unknown'
# En las variables categóricas buscamos detectar valores faltantes codificados como '?', principalmente en 'workclass', 'occupation' y 'native-country' para ambos .csv's (train/val). Estos valores se imputarán explícitamente como una categoría "Unknown" para no descartar registros y permitir que el modelo aprenda, en caso de que exista, un patrón asociado a la ausencia de información.”

cat_cols = [
    "workclass", "education", "marital-status", "occupation",
    "relationship", "race", "sex", "native-country"
]
missing_stats = []

n_train = len(df_train)
n_val   = len(df_val)

for col in cat_cols:
    train_q = (df_train[col] == "?").sum()
    val_q   = (df_val[col] == "?").sum()

    missing_stats.append({
        "columna": col,
        "train_?": train_q,
        "train_%": train_q / n_train * 100,
        "val_?": val_q,
        "val_%": val_q / n_val * 100,
    })

missing_df = pd.DataFrame(missing_stats)
missing_df

In [5]:
# Si existe al menos un '?' en features categóricas, lo reemplazamos por 'Unknown'

for col in cat_cols:
    if (df_train[col] == "?").any() or (df_val[col] == "?").any():
        df_train[col] = df_train[col].replace("?", "Unknown")
        df_val[col]   = df_val[col].replace("?", "Unknown")
        print(f"Reemplazando '?' por 'Unknown' en: {col}")
    else:
        print(f"✔ No se encontraron '?' en: {col}")

print("Target normalizado")

Se verificó la presencia de valores faltantes codificados como '?' en las variables categóricas.
En este dataset en particular no se detectaron observaciones, pero se deja implementado el reemplazo automático que recodifica '?' como 'Unknown' en caso de presentarse en futuros datos.

In [6]:
# Analizamos la cardinalidad de las variables categóricas (sumatoria de categorías únicas)
# Tanto en train como en val, de esta forma podemos ver si existen categorías que sólo aparecen en uno de los conjuntos.
# Esto es importante para anticipar problemas en el modelado con embeddings, ya que las categorías que sólo aparecen en val no tendrán embedding entrenado.

cat_summary = []

for col in cat_cols:
    # categorías únicas por conjunto
    train_uniques = set(df_train[col].unique())
    val_uniques   = set(df_val[col].unique())
    total_uniques = train_uniques.union(val_uniques)

    cat_summary.append({
        "columna": col,
        "card_train": len(train_uniques),
        "card_val": len(val_uniques),
        "card_total": len(total_uniques),
        "solo_en_val": len(val_uniques - train_uniques),
        "solo_en_train": len(train_uniques - val_uniques),
    })

cat_card_df = pd.DataFrame(cat_summary).sort_values("card_total", ascending=False)
cat_card_df

En la variable de alta cardinalidad native-country se observa una discrepancia menor: el conjunto de entrenamiento contiene 41 categorías, mientras que el conjunto de validación contiene 40.

Esta situación no representa un problema para el modelo basado en embeddings, ya que el embedding correspondiente se entrena y simplemente no es utilizado por ninguna muestra de validación.

Para el modelo alternativo sin embeddings (one-hot encoding), la categoría presente solo en train generaría una columna que permanecerá en cero en todo el conjunto de validación, aumentando marginalmente la dimensionalidad del vector sin aportar información útil.

In [7]:
# Revisamos cuál es la categoría que aparece en train pero no en val, para 'native-country'. 
# Todas las demás columnas no presentan categorías exclusivas en ninguno de los dos conjuntos.

# Cuál es?
train_only = train_uniques - val_uniques
train_only = list(train_only)[0]
print(f"Categoría exclusiva de train en 'native-country': {train_only}")

In [8]:
# Cuántas observaciones tienen esa categoría exclusiva de train?
df_train[df_train["native-country"] == train_only].shape[0]

In [9]:
# Cuántas de ellas tienen income >50K?
df_train[df_train["native-country"] == train_only]["income"].value_counts(normalize=True)

La categoría Holand-Netherlands dentro de `native-country` posee una única ocurrencia (≈0.003% del dataset) y no está presente en el conjunto de validación. Dado que su aporte estadístico es nulo y únicamente introduce ruido en las representaciones categóricas (embeddings entrenados con un único sample y columnas OHE esparsas), se decidió eliminar dicha observación del conjunto de entrenamiento.

Este tratamiento resulta más adecuado que crear una imputación específica para un único registro, manteniendo un espacio categórico más limpio y reduciendo parámetros innecesarios en el modelo.

In [10]:
# Aplicamos la decisión de quitar en train esa observación con categoría exclusiva en native-country.

rare_countries = train_uniques - val_uniques
print(f"Eliminando observación categoría rara en 'native-country': {rare_countries}")
df_train = df_train[~df_train["native-country"].isin(rare_countries)].reset_index(drop=True)
# Dejamos comentada la línea de val, ya que no es necesario aplicarla.
# df_val   = df_val[~df_val["native-country"].isin(rare_countries)].reset_index(drop=True)

In [11]:
# Analizamos y clasificamos la cardinalidad de las variables categóricas (sumatoria de categorías únicas)
# Tanto en train como en val, y definimos valores únicos > 10 -> "alta", entre 6 y 9 "media", menor a 6 baja

def categorize_cardinality(n_unique):
    if n_unique >= 10:
        return "alta"
    elif n_unique >= 6:
        return "media"
    else:
        return "baja"

card_stats = []

for col in cat_cols:
    train_uniques = set(df_train[col].unique())
    val_uniques   = set(df_val[col].unique())
    total_uniques = train_uniques.union(val_uniques)

    n_train = len(train_uniques)
    n_val   = len(val_uniques)
    n_total = len(total_uniques)

    card_stats.append({
        "columna": col,
        "card_train": n_train,
        "card_val": n_val,
        "card_total": n_total,
        "cardinalidad": categorize_cardinality(n_total),
        "solo_en_train": list(train_uniques - val_uniques),
        "solo_en_val": list(val_uniques - train_uniques)
    })

cardinality_df = pd.DataFrame(card_stats).sort_values("card_total", ascending=False)
cardinality_df


In [12]:
# Teniendo los datasets limpios, analizamos las frecuencias y porcentajes de cada categoría en train y val para todas las variables categóricas.
# Esto nos permitirá observar si hay categorías dominantes o muy raras, y comparar su distribución entre ambos conjuntos.

# Funciones para análisis y visualización de proporciones de categorías 
def category_proportions(df_train, df_val, col):
    train_counts = df_train[col].value_counts().rename("train_count")
    val_counts   = df_val[col].value_counts().rename("val_count")

    combined = pd.concat([train_counts, val_counts], axis=1).fillna(0)

    combined["train_pct"] = combined["train_count"] / len(df_train) * 100
    combined["val_pct"]   = combined["val_count"] / len(df_val) * 100
    combined["total"]     = combined["train_count"] + combined["val_count"]
    combined["total_pct"] = combined["total"] / (len(df_train) + len(df_val)) * 100

    combined = combined.sort_values("total", ascending=False)

    return combined

def category_proportions_print(df_train, df_val, col):
    tbl = category_proportions(df_train, df_val, col).copy()

    for c in ["train_pct", "val_pct", "total_pct"]:
        tbl[c] = tbl[c].map(lambda x: f"{x:.2f}%")

    return tbl

def plot_category_distribution(df_train, df_val, col):
    combined = category_proportions(df_train, df_val, col)

    plt.figure(figsize=(12,6))
    sns.barplot(
        data=combined.reset_index(),
        x=col,
        y="total_pct",
        order=combined.index
    )
    plt.xticks(rotation=90)
    plt.ylabel("% total (train + val)")
    plt.title(f"Distribución porcentual de '{col}'")
    plt.show()

def plot_train_val_comparison(df_train, df_val, col):
    combined = category_proportions(df_train, df_val, col).reset_index()

    melted = combined.melt(
        id_vars=[col],
        value_vars=["train_pct", "val_pct"],
        var_name="dataset",
        value_name="percentage"
    )

    plt.figure(figsize=(12,6))
    sns.barplot(
        data=melted,
        x=col,
        y="percentage",
        hue="dataset",
        order=combined[col],
        palette="coolwarm"
    )
    plt.xticks(rotation=90)
    plt.ylabel("%")
    plt.title(f"Comparación porcentual train vs val en '{col}'")
    plt.legend(title="")
    plt.show()

In [13]:
# EDA de features categóricas completo

for col in cat_cols:
    print("="*70)
    print(f"Variable categórica: {col}")
    print("="*70)

    # Mostrar tabla resumida
    display(category_proportions_print(df_train, df_val, col).head(10))

    # Gráfico total
    plot_category_distribution(df_train, df_val, col)

    # Comparación train vs val
    plot_train_val_comparison(df_train, df_val, col)


In [14]:
# Analizamos los features numéricos: age, hours-per-week, capital-gain, capital-loss, 

num_cols = ["age", "hours-per-week", "capital-gain", "capital-loss"]

# Histogramas de distribución numérica (excepto capital-gain y capital-loss por su alta asimetría)

for col in ["age", "hours-per-week"]:
    plt.figure(figsize=(7,4))
    sns.histplot(df_train[col], bins=40, kde=True)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()


In [15]:
# capital-gain / capital-loss (skew)

for col in ["capital-gain", "capital-loss"]:

    plt.figure(figsize=(7,4))
    sns.histplot(df_train[col], bins=60)
    plt.title(f"{col} (escala original)")
    plt.show()

    # Sólo valores >0 (para ver al menos algo)
    nonzero = df_train[df_train[col] > 0][col]
    if len(nonzero) > 0:
        plt.figure(figsize=(7,4))
        sns.histplot(nonzero, bins=60)
        plt.title(f"{col} (>0 solamente)")
        plt.show()

In [16]:
# % de ceros en capital-gain y capital-loss

for col in ["capital-gain", "capital-loss"]:
    pct_zero = (df_train[col] == 0).mean() * 100
    print(f"Feature {col}: {pct_zero:.2f}% de valores son cero")


In [17]:
# Boxplots de features numéricos según income

for col in num_cols:
    plt.figure(figsize=(7,4))
    sns.boxplot(x=df_train["income"], y=df_train[col])
    plt.title(f"{col} según income")
    plt.xlabel("Income (0 = <=50K, 1 = >50K)")
    plt.show()

In [18]:
# Correlación de features numéricos vs income

corr = df_train[num_cols + ["income"]].corr()["income"].sort_values(ascending=False)
print("Correlación con income:\n", corr)

## 1. Conclusiones del EDA sobre variables categóricas

Se analizaron las siguientes variables categóricas:  
`workclass`, `education`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `native-country`.

### 1.1. Cardinalidad y estructura

- Se calculó la cardinalidad por variable tanto en `train` como en `val`, verificando además la **cardinalidad total conjunta** y si existían categorías exclusivas de algún conjunto.
- El resultado fue:

  - **Alta cardinalidad (≥10 categorías)**  
    - `education` (~16 categorías)  
    - `occupation` (~14 categorías)  
    - `native-country` (~40 categorías tras la limpieza)

  - **Cardinalidad media (6–9 categorías)**  
    - `workclass` (~7 categorías)  
    - `marital-status` (~7 categorías)  
    - `relationship` (~6 categorías)

  - **Cardinalidad baja (≤5 categorías)**  
    - `race` (~5 categorías)  
    - `sex` (2 categorías)

- Se detectó una categoría presente sólo en `train` para `native-country`:
  - `Holand-Netherlands`, con **1 sola observación** y sin presencia en `val`.
  - Dicha observación correspondía al grupo `<=50K` y representaba ≈0.003 % del dataset.
  - Por su frecuencia insignificante y nulo aporte estadístico, se decidió **eliminar esta observación**, evitando:
    - un embedding entrenado con un único ejemplo, y  
    - una columna OHE completamente dispersa en el modelo sin embeddings.

### 1.2. Distribución de categorías y relación con el target

- Para cada variable categórica se construyeron tablas de frecuencias y **porcentajes** por categoría, tanto para `train` como para `val`, además de la proporción conjunta.
- Se verificó que las distribuciones relativas entre `train` y `val` son consistentes, es decir, no se observan desbalances artificiales introducidos por el split.
- Adicionalmente, se calculó la **proporción de ingresos >50K por categoría** (`P(income = 1 | categoría)`), lo que permitió:
  - identificar categorías fuertemente asociadas a ingresos altos, por ejemplo, ciertas ocupaciones (`Exec-managerial`, `Prof-specialty`), estados civiles (p.ej. `Married-civ-spouse`) o determinadas combinaciones de `relationship`;
  - detectar categorías con tasas muy bajas de >50K (por ejemplo, `Never-married` o ciertos tipos de `workclass`).

Este análisis refuerza la idea de que las variables categóricas contienen una señal fuerte y estructurada respecto del target, pero con patrones **no triviales** que justifican el uso de **embeddings** en las de mayor cardinalidad.

### 1.3. Decisiones preliminares de transformación categórica

A partir del EDA de cardinalidad, distribución y contexto semántico:

- `education`  
  - Tiene una estructura claramente **ordinal** (niveles educativos crecientes).  
  - Se decidió utilizar **ordinal encoding**, preservando el orden lógico de la variable.

- `occupation`  
  - Cardinalidad media-alta (~14), con categorías heterogéneas pero con posibles similitudes (p.ej. profesiones de alto nivel, trabajos de servicios, etc.).  
  - Recomendación: representarla mediante una **capa de embeddings**, permitiendo que el modelo aprenda relaciones latentes entre ocupaciones.

- `native-country`  
  - Cardinalidad alta (~40) y fuerte dominancia de `United-States`, pero con un conjunto largo de países de menor frecuencia.  
  - OHE generaría muchas dimensionalidad con bajo soporte estadístico.  
  - Recomendación: utilizar **embeddings** para esta variable.

- `workclass`, `marital-status`, `relationship`, `race`  
  - Cardinalidad baja/media (5–7), sin estructura de orden natural.  
  - Recomendación: **one-hot encoding (OHE)**.

- `sex`  
  - Variable binaria.  
  - Recomendación: **codificación 0/1** (label encoding simple).

Estas decisiones las implementaremos en el pipeline de Feature Engineering previo al modelo del punto 2.


## 2. Conclusiones del EDA sobre variables numéricas

Se analizaron las variables numéricas:

- `age`
- `hours-per-week`
- `capital-gain`
- `capital-loss`

(Además, `income` fue mapeada a 0/1 y se utilizó como target.)

### 2.1. Distribuciones y outliers

- **Age**  
  - Distribución ligeramente asimétrica hacia la derecha, con mayor concentración en el rango 20–50 años.  
  - El boxplot por `income` muestra que el grupo >50K tiende a tener edades mayores.  
  - Se observan outliers en los extremos superiores, esperables en una muestra grande de población (personas de edad avanzada aún activas laboralmente).

- **Hours-per-week**  
  - Distribución con un pico muy marcado en 40 horas semanales.  
  - Existen valores extremos (60–90 horas), que aparecen como numerosos “outliers” en el boxplot, pero son plausibles desde el punto de vista laboral.  
  - El grupo >50K tiende a trabajar más horas en promedio.

- **Capital-gain** y **capital-loss**  
  - Ambos presentan distribuciones extremadamente **sesgadas**:
    - `capital-gain`: más del ~90 % de los valores son 0.  
    - `capital-loss`: más del ~95 % de los valores son 0.
  - Los pocos valores positivos son muy grandes, generando colas largas y boxplots dominados por outliers.
  - La información relevante no está en la magnitud cruda, sino en:
    - el hecho de tener o no ganancias/pérdidas de capital, y  
    - el orden de magnitud cuando las hay.

No se eliminarán outliers en estas variables, ya que reflejan comportamientos reales en la población y se manejarán mediante transformaciones adecuadas.

### 2.2. Correlación con el target

La correlación de Pearson entre las variables numéricas y `income` (0/1) fue:

- `age`: ~0.24  
- `hours-per-week`: ~0.23  
- `capital-gain`: ~0.22  
- `capital-loss`: ~0.15  

Si bien ninguna variable presenta una correlación lineal extremadamente alta con el target, todas muestran una **señal moderada** y complementaria. En particular:

- `age`, `hours-per-week` y `capital-gain` muestran la relación positiva más clara con `income`.
- `capital-loss` aporta algo de información, pero con fuerza menor.

---

## 3. Implicancias para el Feature Engineering y el modelo con embeddings

A partir del EDA categórico y numérico, se define el siguiente esquema de Feature Engineering, orientado específicamente al entrenamiento de un **modelo MLP con embeddings** para clasificación binaria.

### 3.1. Transformaciones propuestas por variable

| Variable          | Tipo         | Características clave                  | Transformación propuesta                                  |
|-------------------|-------------|----------------------------------------|-----------------------------------------------------------|
| age               | Numérica    | Rango amplio, ligera asimetría        | `StandardScaler`                                          |
| hours-per-week    | Numérica    | Pico en 40h, valores altos plausibles | `StandardScaler`                                          |
| capital-gain      | Numérica    | >90% ceros, skew extremo              | `log1p`, variable binaria `(>0)`, luego escalado         |
| capital-loss      | Numérica    | >95% ceros, skew extremo              | `log1p`, variable binaria `(>0)`, luego escalado         |
| education         | Categórica  | Alta cardinalidad, orden natural      | **Ordinal encoding**                                      |
| occupation        | Categórica  | Cardinalidad media-alta (~14)         | **Embedding**                                             |
| native-country    | Categórica  | Cardinalidad alta (~40)               | **Embedding**                                             |
| workclass         | Categórica  | Cardinalidad media (7)                | **One-Hot Encoding**                                      |
| marital-status    | Categórica  | Cardinalidad media (7)                | **One-Hot Encoding**                                      |
| relationship      | Categórica  | Cardinalidad media (6)                | **One-Hot Encoding**                                      |
| race              | Categórica  | Cardinalidad baja (5)                 | **One-Hot Encoding**                                      |
| sex               | Categórica  | Binaria                               | Codificación 0/1                                          |
| income            | Target      | Binaria (<=50K / >50K)                | Mapeada a 0/1 (se utilizará Binary Cross Entropy Loss)   |

Esta propuesta de FE combina distintas estrategias de codificación, tal como solicita la consigna, y cada decisión está justificada a partir de las distribuciones observadas, la cardinalidad y la semántica de las variables.

### 3.2. Lineamientos para el modelo con embeddings

A partir del Feature Engineering propuesto, el modelo con embeddings se diseñará siguiendo estas pautas:

- **Variables con embeddings**:  
  - `occupation` y `native-country` se representarán con capas de embedding separadas.  
  - La dimensión de cada embedding se definirá en función de la cardinalidad (por ejemplo, usando heurísticas del tipo `d ≈ min(16, round(k^0.5))`, donde `k` es el número de categorías), o reglas similares debidamente fundamentadas.

- **Resto de inputs**:  
  - Variables numéricas escaladas (`age`, `hours-per-week`, versiones logarítmicas de gain/loss + indicadores binarios).  
  - Variables categóricas codificadas con OHE y codificación ordinal.

- **Arquitectura MLP**:  
  - Concatenación de todos los embeddings + features numéricos + OHE en un único vector de entrada.  
  - Varias capas densas (número de capas y neuronas a definir), con funciones de activación no lineales (por ejemplo, ReLU).  
  - Inclusión de **dropout** en las capas ocultas para reducir sobreajuste.

- **Función de costo y optimizador**:  
  - El problema es de **clasificación binaria**, por lo que se utilizará **Binary Cross Entropy** (idealmente `BCEWithLogitsLoss` en PyTorch).  
  - El optimizador será **Adam** (o alguna de sus variantes), tal como indica la consigna.

- **Métricas y evaluación**:  
  - Durante el entrenamiento se registrarán curvas de:
    - **accuracy vs epoch**  
    - **F1 macro vs epoch**  
    tanto para `train` como para `val`.
  - Al finalizar el entrenamiento se reportará:
    - **classification report** de `sklearn` (precision, recall, F1 por clase).  
    - **matriz de confusión absoluta** y **matriz de confusión normalizada por fila** sobre el conjunto de validación.

Con estos planteos cerramos el bloque de EDA y preparamos el Feature Engineering para la implementación del modelo con embeddings.

# Punto B - Diseño y entrenamiento de un modelo con embeddings
- Implementar las transformaciones definidas en el punto anterior e incorporarlas al flujo de entrenamiento.
- El modelo debe incluir, como mínimo, una capa de embedding para representar alguna de las variables categóricas.
- La elección de la dimensión del o los embeddings queda a criterio del estudiante, pero debe estar correctamente fundamentada. Recuerden que no es obligatorio que todos los embeddings tengan la misma dimensión.
- La configuración arquitectónica (número de capas, neuronas por capa, función de activación) es de libre elección.
- Incluir dropout en las capas ocultas de la red.
- Utilizar Adam o alguna de sus variantes como optimizador.
- Seleccionar la función de costo apropiada entre Binary CrossEntropyLoss o Categorical CrossEntropyLoss, según la formulación del problema.
- Mostrar las curvas de accuracy vs epoch y F1 macro vs epoch para los sets de entrenamiento y validación.
- Presentar un classification report generado con sklearn.
- Presentar una matriz de confusión absoluta y otra normalizada por fila, correspondientes al set de validación.

In [19]:
# Feature Engineering propuesto tras el análisis y plateo post-EDA

# Clasificamos features según el análisis exploratorio para el tratamiento previo antes del modelado
# Numéricas
num_raw = ["age", "hours-per-week", "capital-gain", "capital-loss"]

# Categóricas
high_card_embed = ["occupation", "native-country"]   # embeddings
ordinal_cols    = ["education"]                     # ordinal
ohe_cols         = ["workclass", "marital-status", "relationship", "race"]
binary_cols      = ["sex"]                          # label encoding simple (0/1)
target_col       = "income"

In [20]:
# Orden de niveles educativos presente en Adult para encoding ordinal
# El orden lo basamos en el US Census Bureau. Educational Attainment in the United States: 1994.
# (El Census Bureau define explícitamente la escala desde Preschool → Doctorate.)

# Definimos
education_order = [
    "Preschool",
    "1st-4th",
    "5th-6th",
    "7th-8th",
    "9th",
    "10th",
    "11th",
    "12th",
    "HS-grad",
    "Some-college",
    "Assoc-voc",
    "Assoc-acdm",
    "Bachelors",
    "Masters",
    "Prof-school",
    "Doctorate"
]

# Mapeamos educación a valores ordinales
education_mapping = {cat: i for i, cat in enumerate(education_order)}
education_mapping

# Aplicamos el mapeo ordinal a train y val
df_train["education_ord"] = df_train["education"].map(education_mapping)
df_val["education_ord"]   = df_val["education"].map(education_mapping)

In [21]:
# Aplicamos log1p y columnas binarias para gain/loss

# Columnas binarias (tiene o no tiene gain/loss)
df_train["has_capital_gain"] = (df_train["capital-gain"] > 0).astype(int)
df_val["has_capital_gain"]   = (df_val["capital-gain"] > 0).astype(int)

df_train["has_capital_loss"] = (df_train["capital-loss"] > 0).astype(int)
df_val["has_capital_loss"]   = (df_val["capital-loss"] > 0).astype(int)

# Aplicamos la transformación logarítmica
df_train["capital_gain_log"] = np.log1p(df_train["capital-gain"])
df_val["capital_gain_log"]   = np.log1p(df_val["capital-gain"])

df_train["capital_loss_log"] = np.log1p(df_train["capital-loss"])
df_val["capital_loss_log"]   = np.log1p(df_val["capital-loss"])

In [22]:
# Label encoding para 'sex' en train y val
# Female/Male → 0/1
df_train["sex_bin"] = df_train["sex"].map({"Female": 0, "Male": 1})
df_val["sex_bin"]   = df_val["sex"].map({"Female": 0, "Male": 1})

In [23]:
# 'workclass', 'marital-status', 'relationship', 'race': OHE (One-Hot Encoding)

ohe_train = pd.get_dummies(df_train[ohe_cols], prefix=ohe_cols)
ohe_val   = pd.get_dummies(df_val[ohe_cols],   prefix=ohe_cols)

# A pesar de haber analizado que tanto train como val tienen igualdad de categorías en estos 4 features, es decir, no hay diferencias, alineamos las columnas por buena práctica y para, en caso que ocurra, evitar mismatch
ohe_train, ohe_val = ohe_train.align(ohe_val, join="left", axis=1, fill_value=0)

In [24]:
# Preparación final de features numéricos escalados

num_final = [
    "age",
    "hours-per-week",
    "capital_gain_log",
    "capital_loss_log",
    "has_capital_gain",
    "has_capital_loss"
]

scaler = StandardScaler()

df_train_scaled = scaler.fit_transform(df_train[num_final])
df_val_scaled   = scaler.transform(df_val[num_final])

df_train_scaled = pd.DataFrame(df_train_scaled, columns=num_final, index=df_train.index)
df_val_scaled   = pd.DataFrame(df_val_scaled,   columns=num_final, index=df_val.index)

In [25]:
# Construimos las categorías finales para embeddings

# Vocabularios para embeddings
embed_vocabs = {}

for col in high_card_embed:
    cats = sorted(df_train[col].unique())
    embed_vocabs[col] = {cat: i for i, cat in enumerate(cats)}

embed_vocabs

# Mapeamos los índices enteros en train y val
for col in high_card_embed:
    df_train[f"{col}_idx"] = df_train[col].map(embed_vocabs[col]).astype(int)
    df_val[f"{col}_idx"]   = df_val[col].map(embed_vocabs[col]).astype(int)

In [26]:
# Construimos los datasets finales para modelado co Pytorch

# Numéricos
X_train_num = df_train_scaled

# OHE
X_train_ohe = ohe_train

# Ordinal
X_train_ord = df_train[["education_ord"]]

# Binary
X_train_bin = df_train[["sex_bin"]]

# Embedding indices
X_train_emb = df_train[[f"{col}_idx" for col in high_card_embed]]

# Concatenación final para MLP
X_train_full = pd.concat(
    [X_train_num, X_train_ohe, X_train_ord, X_train_bin, X_train_emb], axis=1
)

# El Target
y_train = df_train[target_col].astype(int)

# Aplicamos lo mismo para val
X_val_num = df_val_scaled
X_val_ohe = ohe_val
X_val_ord = df_val[["education_ord"]]
X_val_bin = df_val[["sex_bin"]]
X_val_emb = df_val[[f"{col}_idx" for col in high_card_embed]]

X_val_full = pd.concat(
    [X_val_num, X_val_ohe, X_val_ord, X_val_bin, X_val_emb], axis=1
)
y_val = df_val[target_col].astype(int)

In [27]:
# Chequeo de salud: vamos a validar que el FE tenga consistencia antes de avanzar al modelado 

# Visualizamos la distribución de capital-gain y capital-loss antes y después de la transformación log1p
fig, axes = plt.subplots(2,2, figsize=(12,8))

# capital-gain
sns.histplot(df_train["capital-gain"], bins=50, ax=axes[0,0], kde=True)
axes[0,0].set_title("Original capital-gain")

sns.histplot(df_train["capital_gain_log"], bins=50, ax=axes[0,1], kde=True)
axes[0,1].set_title("Log1p capital-gain")

# capital-loss
sns.histplot(df_train["capital-loss"], bins=50, ax=axes[1,0], kde=True)
axes[1,0].set_title("Original capital-loss")

sns.histplot(df_train["capital_loss_log"], bins=50, ax=axes[1,1], kde=True)
axes[1,1].set_title("Log1p capital-loss")

plt.tight_layout()
plt.show()

In [28]:
# Visualizamos boxplots de las variables numéricas escaladas para verificar la estandarización

plt.figure(figsize=(10,4))
sns.boxplot(data=df_train_scaled)
plt.title("Boxplot de variables numéricas escaladas")
plt.xticks(rotation=45)
plt.show()

In [29]:
# Correlación entre variables numéricas ya transformadas

corr = df_train_scaled.corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlación entre numéricas ya transformadas")
plt.show()

In [30]:
# Visualizamos las nuevas columnas binarias has_capital_gain y has_capital_loss para verificar su distribución

fig, axes = plt.subplots(1,2, figsize=(10,4))

sns.countplot(x=df_train["has_capital_gain"], ax=axes[0])
axes[0].set_title("has_capital_gain")

sns.countplot(x=df_train["has_capital_loss"], ax=axes[1])
axes[1].set_title("has_capital_loss")

plt.show()

In [31]:
# Visualizamos la distribución de los índices para embeddings de alta cardinalidad

for col in high_card_embed:
    plt.figure(figsize=(10,4))
    sns.histplot(df_train[f"{col}_idx"], bins=50)
    plt.title(f"Distribución de índices para embedding: {col}")
    plt.show()

In [32]:
# Verificamos que no haya NaNs en los datasets finales y mostramos sus shapes

print("NaNs en train_full:", X_train_full.isna().sum().sum())
print("NaNs en val_full:", X_val_full.isna().sum().sum())

print("Shape train:", X_train_full.shape)
print("Shape val:", X_val_full.shape)

In [33]:
# Preparamos los datasets de PyTorch Dataset y DataLoader para entrenamiento y validación

# Separamos claramente las features que alimentan directamente al MLP y las que van a embeddings

dense_cols = (
    list(df_train_scaled.columns)      # num_final
    + list(ohe_train.columns)          # OHE
    + ["education_ord", "sex_bin"]     # ordinal + binaria
)

emb_cols = ["occupation_idx", "native-country_idx"]

print(len(dense_cols), dense_cols[:5])
print(emb_cols)

In [34]:
# Definimos la clase Dataset de PyTorch personalizado

class AdultDataset(Dataset):
    def __init__(self, df, dense_cols, emb_cols, y):
        # Dense features
        self.dense = torch.tensor(df[dense_cols].values, dtype=torch.float32)
        # Embedding indices
        self.emb   = torch.tensor(df[emb_cols].values, dtype=torch.long)
        # Target
        self.y     = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.dense[idx], self.emb[idx], self.y[idx]


In [35]:
# Construimos los datasets y dataloaders

# Convertir OHE booleanos a float
ohe_train = ohe_train.astype("float32")
ohe_val   = ohe_val.astype("float32")

train_df_for_model = pd.concat(
    [df_train_scaled, ohe_train, df_train[["education_ord", "sex_bin"]], 
     df_train[emb_cols]],
    axis=1
)

val_df_for_model = pd.concat(
    [df_val_scaled, ohe_val, df_val[["education_ord", "sex_bin"]],
     df_val[emb_cols]],
    axis=1
)

train_dataset = AdultDataset(train_df_for_model, dense_cols, emb_cols, y_train)
val_dataset   = AdultDataset(val_df_for_model,   dense_cols, emb_cols, y_val)

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, drop_last=False)

val_loader   = DataLoader(val_dataset,   batch_size=batch_size,
                          shuffle=False, drop_last=False)

In [36]:
# Modelo con embbeddings
# Definimos una clase que soporte:
#   - uno o más embeddings
#   - o ningún embedding (para el modelo full-OHE de la tercera parte)

class IncomeModel(nn.Module):
    def __init__(self, emb_sizes, n_dense, hidden_dims=[128, 64], dropout=0.3):
        """
        emb_sizes: lista de tuplas (num_categorias, dim_embedding)
        n_dense:   número de features sin embeddings
        """
        super().__init__()

        # Embeddings
        self.emb_layers = nn.ModuleList(
            [nn.Embedding(num_cat, emb_dim) for num_cat, emb_dim in emb_sizes]
        )
        emb_dim_total = sum(emb_dim for _, emb_dim in emb_sizes)

        input_dim = n_dense + emb_dim_total

        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev = h
        layers.append(nn.Linear(prev, 1))  # salida logit
        self.mlp = nn.Sequential(*layers)

    def forward(self, dense_x, emb_x):
        """
        dense_x: (batch_size, n_dense)
        emb_x:   (batch_size, n_emb_fields) con índices enteros
        """
        if len(self.emb_layers) > 0:
            emb_outs = []
            for i, emb in enumerate(self.emb_layers):
                emb_outs.append(emb(emb_x[:, i]))
            emb_cat = torch.cat(emb_outs, dim=1)
            x = torch.cat([dense_x, emb_cat], dim=1)
        else:
            # modelo sin embeddings
            x = dense_x

        logits = self.mlp(x).squeeze(1)
        return logits

In [37]:
# Definimos los tamaños de embeddings basados en la cardinalidad
# Usamos dimensión fija de 8 para ambas variables de alta cardinalidad
# Si bien active-country tiene 41 categorías, 12 dimensiones es el valor razonable para evitar overfitting y evitar el desbalance hacia United States (90%+ de las observaciones)

emb_sizes = [
    (df_train["occupation_idx"].nunique(), 6),
    (df_train["native-country_idx"].nunique(), 12),
]

n_dense = len(dense_cols)

model = IncomeModel(emb_sizes=emb_sizes, n_dense=n_dense,
                    hidden_dims=[128, 64, 32], dropout=0.15)
model

In [38]:
# Configuramos el device, la función de pérdida y el optimizador Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pos_weight = torch.tensor([1.5], device=device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight) # la adecuada para clasificación binaria
# criterion = nn.BCEWithLogitsLoss()   # la adecuada para clasificación binaria
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [39]:
# Construimos el loop de entrenamiento con:
#   - Accuracy por época
#   - F1 Macro por época
#   - Curvas para train/val
#   - Evaluación con classification report
#   - Matriz de confusión absoluta y normalizada

# Accuracy y F1 Macro
def compute_metrics(y_true, logits):
    """
    Calcula accuracy y F1 macro.
    Acepta tensores o numpy arrays.
    Internamente convierte todo a tensores CPU y aplica sigmoid de forma segura.
    """
    # Convertimos a tensores si vienen como numpy
    if isinstance(y_true, np.ndarray):
        y_true = torch.tensor(y_true)

    if isinstance(logits, np.ndarray):
        logits = torch.tensor(logits)

    # Aseguramos tipos correctos en variables
    y_true = y_true.detach().cpu().float()
    logits = logits.detach().cpu().float()

    # Convertimos logits a probabilidades
    probs = torch.sigmoid(logits).numpy()

    # Calculamos las predicciones binarias
    preds = (probs >= 0.5).astype(int)
    y_true_np = y_true.numpy().astype(int)

    # Lsa métricas de desempeño con sklearn
    acc = accuracy_score(y_true_np, preds)
    f1  = f1_score(y_true_np, preds, average="macro")

    return acc, f1

In [40]:
# Loop de entrenamiento

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    history = {
        "train_acc": [], "val_acc": [],
        "train_f1": [],  "val_f1": [],
        "train_loss": [], "val_loss": [],
    }
    
    for epoch in range(epochs):
        model.train()
        train_losses = []
        train_logits_all = []
        train_y_all = []
        
        for dense_x, emb_x, y in train_loader:
            dense_x = dense_x.to(device)
            emb_x   = emb_x.to(device)
            y       = y.to(device)

            optimizer.zero_grad()
            logits = model(dense_x, emb_x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_logits_all.append(logits)
            train_y_all.append(y)

        # Métricas train
        train_logits_all = torch.cat(train_logits_all)
        train_y_all = torch.cat(train_y_all)
        train_acc, train_f1 = compute_metrics(train_y_all, train_logits_all)
        train_loss = sum(train_losses) / len(train_losses)

        # Validación
        model.eval()
        val_losses = []
        val_logits_all = []
        val_y_all = []

        with torch.no_grad():
            for dense_x, emb_x, y in val_loader:
                dense_x = dense_x.to(device)
                emb_x   = emb_x.to(device)
                y       = y.to(device)

                logits = model(dense_x, emb_x)
                loss = criterion(logits, y)

                val_losses.append(loss.item())
                val_logits_all.append(logits)
                val_y_all.append(y)

        # Métricas val
        val_logits_all = torch.cat(val_logits_all)
        val_y_all = torch.cat(val_y_all)
        val_acc, val_f1 = compute_metrics(val_y_all, val_logits_all)
        val_loss = sum(val_losses) / len(val_losses)

        # Guardamos el historial
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)
        history["train_f1"].append(train_f1)
        history["val_f1"].append(val_f1)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Loss: {train_loss:.4f}/{val_loss:.4f} - "
              f"Acc: {train_acc:.4f}/{val_acc:.4f} - "
              f"F1: {train_f1:.4f}/{val_f1:.4f}")

    return history, (val_logits_all, val_y_all)

In [41]:
# Entrenamos el modelo para 50 épocas

num_epochs = 50

history, (val_logits, val_y) = train_model(
    model, train_loader, val_loader,
    criterion, optimizer,
    epochs=num_epochs
)

In [42]:
history_emb = history.copy
logits_emb = val_logits
y_emb = val_y

In [43]:
# Curvas de accuracy y F1 macro vs epoch para los sets de entrenamiento y validación.

# Curva de Accuracy
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history["train_acc"], label="Train Accuracy")
plt.plot(history["val_acc"], label="Val Accuracy")
plt.title("Accuracy por Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Curva de F1 Macro
plt.subplot(1,2,2)
plt.plot(history["train_f1"], label="Train F1 Macro")
plt.plot(history["val_f1"], label="Val F1 Macro")
plt.title("F1 Macro por Epoch")
plt.xlabel("Epoch")
plt.ylabel("F1 Macro")
plt.legend()

plt.tight_layout()
plt.show()

In [44]:
# Classification Report

probs = torch.sigmoid(val_logits).cpu().numpy()
preds = (probs >= 0.5).astype(int)
true  = val_y.cpu().numpy()

print("\n CLASSIFICATION REPORT (CON EMBEDDINGS) \n")
print(classification_report(true, preds, target_names=["<=50K", ">50K"]))

In [45]:
# Matriz de confusión

cm = confusion_matrix(true, preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred <=50K", "Pred >50K"],
            yticklabels=["True <=50K", "True >50K"])
plt.title("Matriz de confusión (absoluta)")
plt.show()

In [46]:
# Matriz de confusión normalizada por fila

cm_norm = confusion_matrix(true, preds, normalize="true")
plt.figure(figsize=(6,5))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=["Pred <=50K", "Pred >50K"],
            yticklabels=["True <=50K", "True >50K"])
plt.title("Matriz de confusión (normalizada)")
plt.show()

In [47]:
# Scatter para modelo CON embeddings

# Aseguramos tensores CPU + probabilidades
probs_emb = torch.sigmoid(val_logits).detach().cpu().numpy()
y_true_emb = val_y.detach().cpu().numpy()

plt.figure(figsize=(7, 6))

# Jitter para dispersar puntos verticalmente
x_jitter = y_true_emb + (np.random.randn(len(y_true_emb)) * 0.02)

plt.scatter(
    x_jitter,
    probs_emb,
    alpha=0.15,
    s=20
)

plt.axhline(0.5, color="red", linestyle="--", linewidth=1.2)

plt.xlabel("Valor real (0 = <=50K, 1 = >50K)")
plt.ylabel("Probabilidad predicha (>50K)")
plt.title("Scatter Real vs Predicho — Modelo con Embeddings")
plt.ylim([-0.02, 1.02])
plt.xlim([-0.2, 1.2])

plt.show()

# Modelo con Embeddings - Análisis, Resultados y Conclusiones

### Contexto:
El objetivo de esta parte siguiendo la consigna fue entrenar un modelo de clasificación binaria (<=50K vs >50K) utilizando embeddings para las variables categóricas de alta cardinalidad.

Las variables transformadas mediante embeddings fueron:
    - occupation (14 categorías)
    - native-country (41 categorías)

El resto de las variables categóricas se codificaron con OHE u ordinal encoding según el caso y justificaciones ya mencionadas.

### Arquitectura del modelo y estrategia de búsqueda

El proceso de modelado se estructuró en dos etapas claramente separadas:

1) Ajuste de la arquitectura base del MLP (antes de tocar tamaño de embeddings)
Se experimentó primero con los hiperparámetros del cuerpo denso del modelo:
- Capas ocultas: se probaron configuraciones
    - 128 → 64
    - 128 → 64 → 32
- Dropout: se ajustó entre 0.30 → 0.15, buscando balance entre underfitting/overfitting.
- pos_weight: se calibró en el rango 1.3–1.6 para compensar el desbalance.
- batch_size: se probaron 512 → 128, favoreciendo una señal de gradiente más estable.
- Épocas: 100 → 70 → 50, evaluando convergencia vs. sobreajuste.
- Optimizador: Adam, función de activación ReLU, Loss BCEWithLogitsLoss.

Esta etapa sirvió para fijar una arquitectura estable, convergente y comparativamente robusta.
Recién cuando estuvo firme, pasamos a la etapa 2.

2) Ajuste del tamaño de los embeddings
Con la arquitectura consolidada, se testeó la sensibilidad del modelo al tamaño de cada embedding:
- occupation_idx: 8 → 6
- native_country_idx: 8 → 10 → 12

Este fine tuning permitió explorar si un embedding más comprimido mejoraba la generalización en categorías ruidosas y si un embedding más amplio capturaba mejor la alta cardinalidad del país de origen.

#### Evolución del tuning de embeddings

Se realizaron cuatro ejecuciones principales con variaciones en los tamaños de embedding:

| Ejecución     | occupation_emb | native_country_emb | F1 (>50K) | Recall (>50K) | Macro F1 |
|---------------|:----------------:|:--------------------:|:-----------:|:----------------:|:----------:|
| Exec 1        | 8              | 8                  | 0.66      | 0.58–0.63      | 0.78     |
| Exec 2        | 6              | 10                 | 0.67      | ~0.70          | 0.78–0.79|
| Exec 3        | 6              | 10                 | 0.68      | ~0.70          | 0.79     |
| **Exec Final**| **6**          | **12**             | **0.68**  | **0.72**       | **0.79** |


#### Conclusión del tuning
- occupation funciona mejor con un embedding más comprimido (6 dimensiones).
- native-country mejora ligeramente al subir a 12 dimensiones, pero muestra señales claras de que ese es su “techo natural” antes de introducir ruido o sobreajuste.
- La combinación occupation=6 / native-country=12 fue la que produjo mejor recall de la clase minoritaria y curvas más estables.


### Curvas de aprendizaje

Las curvas de accuracy y F1 macro mostraron:
- Convergencia suave, sin picos o saltos pronunciados.
- Overfitting bajo debido al dropout del 15% y el pos_weight balanceado.
- Mejor estabilidad en validación respecto a versiones anteriores del modelo.
- Rendimiento máximo alcanzado muy cerca de la época 30–35 siendo 50 épocas es un buen punto de corte.

### Classification Report – Validación

| Clase         | Precision | Recall | F1-score | Support |
|---------------|:-----------:|:--------:|:----------:|:---------:|
| <=50K         | 0.91      | 0.87   | 0.89     | 11360   |
| >50K          | 0.65      | 0.72   | 0.68     | 3700    |
| **Accuracy**  | —         | —      | **0.84** | **15060** |
| **Macro Avg** | 0.78      | 0.80   | 0.79     | 15060   |
| **Weighted Avg** | 0.84   | 0.84   | 0.84     | 15060   |


- Accuracy ≈ 84% → lo que indica que el modelo logra capturar una parte relevante de la relación entre las variables demográficas y el nivel de ingreso, sin caer en sobreajuste..
- F1-score (minoritaria) = 0.68 → sólido para un problema altamente desbalanceado.
- Recall(minoritaria) = 0.72 → el mejor obtenido entre todas las configuraciones.
- Macro-F1 = 0.79 → buen equilibrio entre clases.

## Conclusiones del modelo con embeddings

a) Los embeddings demostraron ser la mejor representación para variables con alta cardinalidad y semántica débil, evitando mucha dimensionalidad y capturando relaciones internas no lineales.

b) La combinación occupation_idx=6 y native_country_idx=12 fue la más eficaz, logrando el mejor recall para la clase >50K.

c) El modelo converge de forma estable, sin sobreajuste fuerte, y mantiene un balance sólido entre todas las métricas relevantes.

d) Este modelo queda seleccionado como el “modelo base con embeddings” para la comparación final contra el modelo alternativo sin embeddings + OHE de la parte siguiente.


# Punto C - Diseño y entrenamiento de un modelo sin embeddings

- Entrenar un segundo modelo, aplicando one-hot encoding a todas las variables que en el punto b) fueron representadas mediante embeddings.
- Mantener exactamente la misma arquitectura del modelo anterior: igual número de capas, mismas neuronas, mismas funciones de activación y la misma probabilidad de dropout.
- Presentar las mismas métricas, visualizaciones y reportes que en el modelo con embeddings.

In [48]:
# Modelo sin embeddings (solo OHE), Mantener exactamente la misma arquitectura del modelo con embeddings:
# - Capas: 128 → 64 → 32
# - Dropout: 0.15
# - Optimizador: Adam
# - Función de pérdida: BCEWithLogitsLoss(pos_weight)
# - Batch_size: 128
# - Cantidad de épocas: 50

# Variables categóricas: workclass, education, marital-status, occupation, relationship, race, sex, native-country
# Las codificamos con OHE clásico

# Variables numéricas: age, hours-per-week, capital_gain_log, capital_loss_log, has_capital_gain, has_capital_loss, education_ord, sex_bin
# Quedan igual con el FE aplicado para el modelo con embeddings

# Income queda con también igual con 0/1 (<=50K / >50K)

# Definimos variables categóricas (las mismas que antes)
cat_cols = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country"
]

# Definimos variables numéricas (ídem)
num_cols = [
    "age",
    "hours-per-week",
    "capital_gain_log",
    "capital_loss_log",
    "has_capital_gain",
    "has_capital_loss",
    "education_ord",
    "sex_bin"
]

# OHE en train
train_noemb = pd.get_dummies(df_train[cat_cols], drop_first=False)
train_noemb = pd.concat([df_train[num_cols], train_noemb], axis=1)

# OHE en val
val_noemb = pd.get_dummies(df_val[cat_cols], drop_first=False)
val_noemb = pd.concat([df_val[num_cols], val_noemb], axis=1)

# AlineaMOS columnas
train_noemb, val_noemb = train_noemb.align(val_noemb, join="left", axis=1, fill_value=0)

# Extraemos targets
y_train = df_train["income"].values.astype("float32")
y_val = df_val["income"].values.astype("float32")

# Aseguramos un solo dtype numérico para todas las columnas
train_noemb = train_noemb.astype("float32")
val_noemb   = val_noemb.astype("float32")

# Convertimos X a tensores
X_train = torch.tensor(train_noemb.values, dtype=torch.float32)
X_val   = torch.tensor(val_noemb.values,   dtype=torch.float32)

# Definimos la dimensión de entrada del MLP sin embeddings
input_dim_noemb = X_train.shape[1]
input_dim_noemb

In [49]:
# X e y para el modelo sin embeddings (solo features densos + OHE)
X_train_noemb = X_train.clone()
X_val_noemb   = X_val.clone()

y_train_noemb = torch.tensor(y_train, dtype=torch.float32)
y_val_noemb   = torch.tensor(y_val,   dtype=torch.float32)

# Dataset
train_dataset_noemb = TensorDataset(X_train_noemb, y_train_noemb)
val_dataset_noemb   = TensorDataset(X_val_noemb,   y_val_noemb)

# DataLoaders (mantenemos batch_size = 128 como en la mejor exec del modelo con embeddings)
batch_size = 128

train_loader_noemb = DataLoader(
    train_dataset_noemb,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False
)

val_loader_noemb = DataLoader(
    val_dataset_noemb,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

print("Input dim sin embeddings:", input_dim_noemb)

In [50]:
# Definimos la clase del modelo MLP idéntico al anterior, pero sis embeddings

class IncomeMLP_NoEmb(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.15),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.15),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.15),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

In [51]:
# Entrenamos el modelo sin embeddings

model_noemb = IncomeMLP_NoEmb(input_dim_noemb).to(device)

criterion = nn.BCEWithLogitsLoss(
    pos_weight=torch.tensor([pos_weight], device=device)
)
optimizer = torch.optim.Adam(model_noemb.parameters(), lr=1e-3)

history_noemb = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": [],
    "train_f1": [],
    "val_f1": [],
}

for epoch in range(1, num_epochs + 1):
    # Train
    model_noemb.train()
    running_loss = 0.0
    all_logits_train = []
    all_y_train = []

    for X_batch, y_batch in train_loader_noemb:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model_noemb(X_batch).squeeze(1)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        all_logits_train.append(logits.detach().cpu())
        all_y_train.append(y_batch.detach().cpu())

    train_loss = running_loss / len(train_loader_noemb.dataset)
    all_logits_train = torch.cat(all_logits_train)
    all_y_train = torch.cat(all_y_train)

    train_acc, train_f1 = compute_metrics(
                        all_y_train,
                        all_logits_train
    )

    # Val
    model_noemb.eval()
    val_running_loss = 0.0
    all_logits_val = []
    all_y_val = []

    with torch.no_grad():
        for X_batch, y_batch in val_loader_noemb:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model_noemb(X_batch).squeeze(1)
            loss = criterion(logits, y_batch)

            val_running_loss += loss.item() * X_batch.size(0)
            all_logits_val.append(logits.cpu())
            all_y_val.append(y_batch.cpu())

    val_loss = val_running_loss / len(val_loader_noemb.dataset)
    all_logits_val = torch.cat(all_logits_val)
    all_y_val = torch.cat(all_y_val)
    val_acc, val_f1 = compute_metrics(
        all_y_val, all_logits_val
    )

    history_noemb["train_loss"].append(train_loss)
    history_noemb["val_loss"].append(val_loss)
    history_noemb["train_acc"].append(train_acc)
    history_noemb["val_acc"].append(val_acc)
    history_noemb["train_f1"].append(train_f1)
    history_noemb["val_f1"].append(val_f1)

    print(
        f"Epoch {epoch}/{num_epochs} - "
        f"Loss: {train_loss:.4f}/{val_loss:.4f} - "
        f"Acc: {train_acc:.4f}/{val_acc:.4f} - "
        f"F1: {train_f1:.4f}/{val_f1:.4f}"
    )
# Guardamos variables para classification report y plots
val_logits_noemb = all_logits_val.clone()
val_y_noemb      = all_y_val.clone()

In [52]:
# Modelo sin embeddings - curvas de accuracy y F1 macro vs epoch para los sets de entrenamiento y validación.

x = range(1, num_epochs + 1)

plt.figure(figsize=(14,5))

# Curva de Accuracy
plt.subplot(1,2,1)
plt.plot(x, history_noemb["train_acc"], label="Train Acc")
plt.plot(x, history_noemb["val_acc"], label="Val Acc")
plt.title("Accuracy vs Epoch (Sin Embeddings)")
plt.xlabel("Epoch"); plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(x, history_noemb["train_f1"], label="Train F1 Macro")
plt.plot(x, history_noemb["val_f1"], label="Val F1 Macro")
plt.title("F1 Macro vs Epoch (Sin Embeddings)")
plt.xlabel("Epoch"); plt.ylabel("F1 Macro")
plt.legend()
plt.grid(True)

plt.show()

In [53]:
# Classification Report

probs_noemb = torch.sigmoid(val_logits_noemb).detach().cpu().numpy()
preds_noemb = (probs_noemb >= 0.5).astype(int)
y_true_noemb = val_y_noemb.cpu().numpy()

print("\n CLASSIFICATION REPORT (SIN EMBEDDINGS) \n")
print(classification_report(y_true_noemb, preds_noemb, digits=4))

In [54]:
# Matriz de confusión

cm = confusion_matrix(y_true_noemb, preds_noemb)

plt.figure(figsize=(7,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusión — Modelo sin Embeddings")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

In [55]:
# Matriz de confusión normalizada por fila

cm_norm = confusion_matrix(y_true_noemb, preds_noemb, normalize='true')

plt.figure(figsize=(7,5))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Greens")
plt.title("Matriz Normalizada — Modelo sin Embeddings")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

In [56]:
# Scatter de predicción Vs. real sin embeddings

plt.figure(figsize=(7, 6))
plt.scatter(
    y_true_noemb + np.random.normal(0, 0.02, size=len(y_true_noemb)),  # jitter eje X
    probs_noemb,
    alpha=0.25,
    s=12
)

plt.axhline(0.5, color="red", linestyle="--", linewidth=1)
plt.yticks([0, 0.25, 0.5, 0.75, 1.0])
plt.xlabel("Valor real (0 = <=50K, 1 = >50K)")
plt.ylabel("Probabilidad predicha (>50K)")
plt.title("Scatter Real vs Predicho — Modelo sin Embeddings")
plt.grid(alpha=0.15)
plt.show()

In [57]:
# Histograma de las diferencias entre modelos (EMB vs. NOEMB)

probs_emb   = torch.sigmoid(val_logits).detach().cpu().numpy()
probs_noemb = torch.sigmoid(val_logits_noemb).detach().cpu().numpy()

diff = np.abs(probs_emb - probs_noemb)

plt.figure(figsize=(7,5))
plt.hist(diff, bins=40, edgecolor="k", alpha=0.7)
plt.title("Diferencia absoluta de probabilidad\nEmbeddings vs No Embeddings")
plt.xlabel("|p_emb - p_noemb|")
plt.ylabel("Frecuencia")
plt.grid(alpha=0.2)
plt.show()

print("Dif media:", diff.mean())
print("Dif p90 :", np.percentile(diff, 90))
print("Dif max :", diff.max())

## Tabla comparativa de los 2 modelos, con y sin embeddings

### Comparación de desempeño entre modelos

| Modelo                 | Accuracy (Val) | F1 Macro (Val) | Loss (Val) |
|------------------------|:----------------:|:----------------:|:------------:|
| **Con Embeddings**     | 0.8352         | 0.7860         | 0.4523     |
| **Sin Embeddings (OHE)** | 0.8445         | 0.7889         | 0.4082     |


## Parte D - Conclusiones y observaciones finales

A partir de los resultados obtenidos en ambos enfoques —MLP con embeddings y MLP tradicional con variables categóricas OHE— se pueden sacar algunas conclusiones:

1. Desempeño predictivo

- En términos de métricas finales, no se observan diferencias significativas entre los dos modelos.
    Ambos alcanzan:
    - Accuracy ≈ 0.84
    - F1 Macro ≈ 0.79
    - Un patrón de aprendizaje muy similar a lo largo de las épocas.

Podemos decir que ambos modelos tienen desempeños similares, ninguno saca una ventaja clara sobre el otro.

2. ¿Por qué los embeddings no dieron una mejora marcada?

- Los embeddings suelen distinguirse en desempeño cuando:
    - Categorías muy numerosas o tienen una estructura semántica útil.
    - Hay dimensionalidad excesiva y el OHE genera mucha cantidad de fratures adicionales.
    - Hay interacciones latentes entre categorías.

- En este dataset, sin embargo:
    - Las columnas categóricas son relativamente chicas (salvo native-country, que igual aporta poco porque el 90% cae en “United-States”).
    - El OHE no genera una dimensionalidad elevada, por lo que el MLP sin embeddings no se ve afectado.
    - Muchas categorías tienen baja cardinalidad (sex, race, relationship, marital-status…), lo que limita la capacidad de los embeddings para descubrir “relaciones internas” entre ellas.

    Resultado en este caso: los embeddings no aprenden nada suficientemente distinto como para superar al OHE tradicional.

3. Interpretación del histograma de diferencias

- El análisis de diferencias entre probabilidades predichas (modelo con vs sin embeddings) mostró:
    - Diferencia media: 0.056 → los modelos predicen muy parecido.
    - Percentil 90: 0.15 → incluso en los casos donde divergen, no es sustantivo.
    - Picos extremos aislados → algunos outliers donde uno de los modelos diverge un poco más que el otro, pero no afectan la métrica global.

    Esto confirma lo anterior: Los dos modelos ven prácticamente lo mismo y toman decisiones casi idénticas.

4. ¿Cuál modelo conviene elegir?

    Depende más del gusto y el uso que del rendimiento:

- Sin embeddings (OHE)
    - Más directo
    - Menos complejo, más fácil de explicar
    - Métricas marginalmente mejores

- Con embeddings
    - Más elegante
    - Más escalable si el día de mañana se agregan más columnas categóricas con alta cardinalidad

    Con este dataset, el modelo OHE resulta más simple y alcanza un rendimiento igual o apenas mejor, por lo que termina siendo la mejor opción práctica.

5. Conclusión final

    Ambos enfoques funcionan bien, pero el modelo sin embeddings termina ganando por puntos, más por simplicidad y estabilidad que por diferencia en performance.
    La estructura del dataset favorece a un MLP clásico con OHE: categorías pequeñas, país dominante, y una dimensionalidad que no exige al modelo aprender a representar una variable categórica con menos dimensiones que las que tendría el OHE.